# Data Collection

## Objectives:

1. **Load and Preprocess Cheery Leaves Images:**
    - Read and load cherry leaf images from provided dataset.
    - Perform any necessary preprocessing steps, such as resizing and normalization.

2. **Annotate images with Labels:**
    - Assign labels to each image indicating, whether the cherry leaf is healthy or contains powdery milldew.

## Inputs:

1. **Dataset:**
   - A dataset containing cherry leaf images.
   - The dataset should have a structure where images are organized in folders or directories.
   - Each image should be associated with a specific class or label (healthy or mildew).

2. **Paths:**
   - Path to the local dataset on your computer (`dataset_path`).
   - Path to the Gitpod workspace (`workspace_path`).

## Outputs:

1. **Processed Images:**
   - The preprocessed cherry leaf images ready for training.

2. **Labels:**
   - A mapping between image filenames and their corresponding labels (healthy or mildew).

## Additional Comments:

   - no additional comments




---

# Import packages

In [ ]:
%pip install -r /workspaces/Portfolio-project-5-Milldew-detection-in-Cherry-Leaves/requirements.txt

: 